In [7]:
include("dsm.jl")
import dsm
import PyPlot

In [8]:
f_s = 1e6;
T_s = 1/f_s;

n_fft = 2^20;
n_ini = 2^16;
k = 71;

f_i = f_s/n_fft*k;
a_i = .5;
b_i = 0;

n_i = 0;

t = 0:T_s:T_s*(n_fft+n_ini-1);
inp = a_i*sin.(2*pi*f_i*t) + b_i + 2*n_i*(rand(size(t))-.5);

int_1 = zeros(size(t));
int_2 = zeros(size(t));
q_0   = zeros(size(t));

q_0[1] = 0;

for i = 2:length(inp)
    int_1[i] = int_1[i-1] + (inp[i] - q_0[i-1]);
    int_2[i] = int_2[i-1] + (int_1[i] - q_0[i-1]);
    q_0[i] = (int_2[i] > 0)*2-1;
end

q = q_0[end-n_fft+1:end];

q_snr = dsm.snr(q, f_i, f_s/(2*256), f_s);

w = DSP.Windows.kaiser(n_fft, 20);
wf = mean(w);
Y_inp = fft(q.*w)/n_fft/wf;

f = linspace(0, f_s/2, (n_fft/2+1));

PyPlot.semilogx(f, 20*log10.(2*abs.(Y_inp[1:trunc(Int64, n_fft/2+1)])), label=@sprintf("snr = %.2f", q_snr));
PyPlot.hold(true);
PyPlot.semilogx(f[1:trunc(Int64, n_fft/(2*256))], 20*log10.(2*abs.(Y_inp[1:trunc(Int64, n_fft/(2*256))])))
PyPlot.grid(loc=2);
PyPlot.hold(false);
PyPlot.grid(true);
PyPlot.xlabel("frequency [Hz]");
PyPlot.ylabel("amplitude");



LoadError: [91mPyError (ccall(@pysym(:PyObject_Call), PyPtr, (PyPtr, PyPtr, PyPtr), o, arg, kw)) <class 'AttributeError'>
AttributeError('Unknown property loc',)
  File "/home/cgurleyuk/.virtualenvs/local/lib/python3.6/site-packages/matplotlib/pyplot.py", line 3716, in grid
    ret = gca().grid(b=b, which=which, axis=axis, **kwargs)
  File "/home/cgurleyuk/.virtualenvs/local/lib/python3.6/site-packages/matplotlib/axes/_base.py", line 2498, in grid
    self.xaxis.grid(b, which=which, **kwargs)
  File "/home/cgurleyuk/.virtualenvs/local/lib/python3.6/site-packages/matplotlib/axis.py", line 1421, in grid
    tick.gridline.update(kwargs)
  File "/home/cgurleyuk/.virtualenvs/local/lib/python3.6/site-packages/matplotlib/artist.py", line 847, in update
    for k, v in props.items()]
  File "/home/cgurleyuk/.virtualenvs/local/lib/python3.6/site-packages/matplotlib/artist.py", line 847, in <listcomp>
    for k, v in props.items()]
  File "/home/cgurleyuk/.virtualenvs/local/lib/python3.6/site-p

In [9]:
cic_r = 256;
cic_m = 1;
cic_n = 3;

cic_int_1 = zeros(size(q_0));
cic_int_2 = zeros(size(q_0));
cic_int_3 = zeros(size(q_0));
cic_dec   = zeros(length(q_0)/cic_r+1);
cic_dif_1 = zeros(length(q_0)/cic_r+1);
cic_dif_2 = zeros(length(q_0)/cic_r+1);
cic_dif_3 = zeros(length(q_0)/cic_r+1);
t_dec     = zeros(length(q_0)/cic_r+1);

c = 0;
k = cic_m;
for i = 2:length(q_0)
    cic_int_1[i] = cic_int_1[i-1] + q_0[i];
    cic_int_2[i] = cic_int_2[i-1] + cic_int_1[i];
    cic_int_3[i] = cic_int_3[i-1] + cic_int_2[i];
    c = c + 1;
    if c == cic_r
        c = 0;
        k = k + 1;
        t_dec[k] = t[i];
        cic_dec[k] = cic_int_3[i];
        cic_dif_1[k] = cic_dec[k] - cic_dec[k-cic_m];
        cic_dif_2[k] = cic_dif_1[k] - cic_dif_1[k-cic_m];
        cic_dif_3[k] = cic_dif_2[k] - cic_dif_2[k-cic_m];
    end
end

k = (cic_r*cic_m)^cic_n;
cic_out = cic_dif_3[trunc(Int64, end-n_fft/cic_r+1):end]/k;

In [10]:
cic_n_fft = trunc(Int64, n_fft/cic_r);
w = DSP.Windows.kaiser(cic_n_fft, 20);
wf = mean(w);
Y_cic = fft(cic_out.*w)/cic_n_fft/wf;

f_cic = linspace(0, f_s/2/cic_r, (cic_n_fft/2+1));

cic_snr = dsm.snr(cic_out, f_i, f_s/(2*256), f_s/256);

PyPlot.semilogx(f, 20*log10.(2*abs.(Y_inp[1:trunc(Int64, n_fft/2+1)])));
PyPlot.hold(true);
PyPlot.semilogx(f[1:trunc(Int64, n_fft/(2*256))], 20*log10.(2*abs.(Y_inp[1:trunc(Int64, n_fft/(2*256))])), label=@sprintf("snr = %.2f", q_snr));
PyPlot.semilogx(f_cic, 20*log10.(2*abs.(Y_cic[1:trunc(Int64, cic_n_fft/2+1)])), label=@sprintf("snr = %.2f", cic_snr));
PyPlot.legend(loc=2)
PyPlot.hold(false);
PyPlot.grid();
PyPlot.xlabel("frequency [Hz]");
PyPlot.ylabel("amplitude [dB]");

LoadError: [91mUndefVarError: snrx not defined[39m